In [1]:
from database import Database
from transformators.keyword_transformator import KeywordTransformator
#from transformators.author_transformator import AuthorTransformator
from db_credentials	import dwh_db_connection_params
from database import Database
from variables import sourcepath
import os
import pandas as pd

In [56]:
from_authors=pd.read_csv(os.path.join(sourcepath, 'authors.csv'))
from_references=pd.read_csv(os.path.join(sourcepath, 'unique_references.csv'))

In [57]:
from_references.head(2)

,citekey,authors,year,title,journal,volume,issue,pages,source_type,publisher,place,editor,monograph_title,note
0,autio_e_information_exposure_opportunity_evalu...,"Autio, E.; Dahlander, L.; Frederiksen, L.",2013,"Information Exposure, Opportunity Evaluation, ...",Acad. Manag. J,56,5,1348-1371,article,NaN,NaN,NaN,NaN,NaN
1,barrett_m_creating_value_in_online_communities...,"Barrett, M.; Oborn, E.; Orlikowski, W.",2016,Creating Value in Online Communities: The Soci...,Inform. Syst. Res,27,4,704-723,article,NaN,NaN,NaN,NaN,NaN


# Transfrom the references to df with the columns "surname" and "lastname"

In [191]:

def split_into_lists_of_two_strings(names):
    """Takes list of names and splits it into sublists of length 2.
    If the length of the split and flattened list is uneven, the last string is dropped.
    
    Args:
        names(list): list of names of multiple authors.

    Returns:
        list of sublists with each sublist containing surname and firstname of one author.
     """
    #split strings upon ' '  and flatten the resulting list of sublists
    names2=[item for sublist in [s.split(' ') for s in names] for item in sublist]
    all=[]
    while len(names2) > 2:
        auth=names2[:2]
        all.append(auth)
        names2=names2[2:]
    return all



#create a new dataframe of authors where each author gets an own row and empty rows are discarded
ref_aut=pd.Series(from_references['authors'].str.split('; ').explode(ignore_index=True).str.split(', ').dropna())

#remove the 'Van' if existing in strings that are longer tan 2, as checks have shown these are most probably parsing errors
ref_aut.apply(lambda l: (l.remove('Van') if 'Van' in l else l) if len(l)>2 else l)

#df_ex_aut.apply(lambda l: (split_into_lists_of_two_strings([item for sublist in [s.split(' ') for s in l] for item in sublist])) if len(l)>2 else l)

#keep only the rows with a length of 2 now:
keep=ref_aut[ref_aut.str.len()==2]

#those that are longer must be changed
change1=ref_aut[ref_aut.str.len()>2]
#first split every string in list from each other, then split into sublist pairs of two
change1=change1.apply(lambda l: split_into_lists_of_two_strings(l))
#and append now corrected series again to keep
keep.append(change1.explode())

#those that are shorter must be changed
change2=ref_aut[ref_aut.str.len()<2]
#those that are shorter than 2 letters can be dropped
change2=change2[change2.explode().str.len()>2]
#many rows contain names that are just not separated by comma and therefore not recognized. lets split them into sublist of two strings each
change2=change2.apply(lambda l: split_into_lists_of_two_strings(l))
#unnest lists to new rows
change2=change2.explode()
#keep only those that contain two strings
change2=change2[change2.str.len()==2]
#and append the transformed rows to keep
keep.append(change2)

#generate a new dataframe
ref_aut_df=pd.DataFrame(keep)
#split up authors column into surname and firstname
ref_aut_df[['surname', 'firstname']]=pd.DataFrame(ref_aut_df.authors.tolist(), index=ref_aut_df.index)
#keep only those two columns
reference_authors=ref_aut_df[['surname', 'firstname']]

#last, remove duplicate authors
reference_authors.drop_duplicates(inplace=True, ignore_index=True)

reference_authors.head(3)


,surname,firstname
0,Autio,E.
1,Dahlander,L.
2,Frederiksen,L.


In [203]:
from_authors.head(2)

,article_id,author_position,fullname,surname,firstname,middlename,email,departments,institutions,countries
0,1,1,"Yan, Kevin",Yan,Kevin,NaN,NaN,NaN,Dalton State College,United States
1,1,2,"Leidner, Dorothy E.",Leidner,Dorothy,E.,NaN,NaN,Baylor University and Lund University,United States
